# Load packages

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import seaborn as sns
import missingno as msno
import warnings

from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss

# Configuration

In [ ]:
# Korean font
from matplotlib import font_manager, rc
try:
    font_path = "C:/Windows/Fonts/malgun.TTF"
    Kfont = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=Kfont)
except:
    pass

# Fix minus presentation
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
# Ignore warings
warnings.filterwarnings('ignore')

# Custom functions

In [ ]:
def to_date_inv(df):
    try:
        df["날짜"] = pd.to_datetime(df["날짜"], format = "%Y년 %m월 %d일")
        df = df.sort_values(by = "날짜", ascending=True)
        df.set_index("날짜", inplace=True)
    except:
        df['일자'] = pd.to_datetime(df['일자'], format='')
        df = df.sort_values(by = "일자", ascending=True)
        df = df.rename(columns={'일자': '날짜'})
        df.set_index("날짜", inplace=True)

    return df

def to_numeric_inv(df):
    try:
        df["종가"] = df["종가"].replace(",", "")
        df["종가"] = pd.to_numeric(df["종가"])
    except:
        df["종가"] = df["종가"].str.replace(",", "")
        df["종가"] = pd.to_numeric(df["종가"])

    return df

def dtype_chg(df: pd.DataFrame) -> pd.DataFrame:
    return to_date_inv(to_numeric_inv(df))

# Load data

In [ ]:
# Get carbon credit data
path_kau19 = 'data/external_data/KAU19-22/KAU19.csv'
path_kau20 = 'data/external_data/KAU19-22/KAU20.csv'

# Get data path from local directory
path_brent_crude_future = 'data/external_data/Features/브렌트유 선물 내역.csv'
path_natural_gas = 'data/external_data/Features/천연가스 선물 내역.csv'
path_iron = 'data/external_data/Features/철광석 내역.csv'
path_coal_future = 'data/external_data/Features/Newcastle Coal Futures 내역.csv'
path_usd_krw = 'data/external_data/Features/USD_KRW 내역.csv'
path_wti_crude_future = 'data/external_data/Features/WTI유 선물 내역.csv'

In [ ]:
# Load carbon credit dataset
df_kau19 = pd.read_csv(path_kau19, encoding='euc-kr', index_col=0)
df_kau20 = pd.read_csv(path_kau20, encoding='euc-kr', index_col=0)

# Load dataset
df_brent_crude_fut = pd.read_csv(path_brent_crude_future).dropna()
df_natural_gas = pd.read_csv(path_natural_gas).dropna()
df_iron = pd.read_csv(path_iron, encoding='utf-8').dropna()
df_coal_fut = pd.read_csv(path_coal_future, encoding='utf-8').dropna()
df_usd_krw = pd.read_csv(path_usd_krw).dropna()
df_wti_crude_fut = pd.read_csv(path_wti_crude_future).dropna()

In [ ]:
# Check KAU data set
df_kau19

In [ ]:
# Check brent crude oil future data
df_brent_crude_fut

# Preprocessing & Visualization

In [ ]:
# Simple dtype change : date to datetime & Close price comma fix
df_kau19 = dtype_chg(df_kau19)
df_kau20 = dtype_chg(df_kau20)

df_brent_crude_fut = dtype_chg(df_brent_crude_fut)
df_natural_gas = dtype_chg(df_natural_gas)
df_iron = dtype_chg(df_iron)
df_coal_fut = dtype_chg(df_coal_fut)
df_usd_krw = dtype_chg(df_usd_krw)
df_wti_crude_fut = dtype_chg(df_wti_crude_fut)

In [ ]:
# Concat close price of all time series data
df_tot = pd.concat(
    [
        df_kau19['종가'],
        df_kau20['종가'],
        df_brent_crude_fut['종가'],
        df_natural_gas['종가'],
        df_iron['종가'],
        df_coal_fut['종가'],
        df_usd_krw['종가'],
        df_wti_crude_fut['종가'],
],
axis=1)

tot_col_nm = [
        'kau19',
        'kau20',
        'brent_crude_fut',
        'natural_gas',
        'iron',
        'coal_fut',
        'usd_krw',
        'wti_crude_fut',
    ]

df_tot.columns = tot_col_nm

df_tot.head()
df_kau19.index

In [ ]:
# Split total DataFrame by KAU
df_19 = df_tot.drop(columns=['kau20'])
df_20 = df_tot.drop(columns=['kau19'])

In [ ]:
# Weekly temporal smoothing
df_19 = df_19.resample('w').mean()
df_20 = df_20.resample('w').mean()

In [ ]:
# Assign temporal restricition
df_19 = df_19[(df_19.index > pd.Timestamp('2018-12-31')) & (df_19.index < pd.Timestamp('2021-01-01'))]
df_20 = df_20[(df_20.index > pd.Timestamp('2019-12-31')) & (df_20.index < pd.Timestamp('2022-01-01'))]

In [ ]:
# Check temporal restriction and smoothing
df_20

In [ ]:
# Check missing value of each column and drop missing
def fill_missing_and_visualize(df: pd.DataFrame, df_nm=None, visualize=True) -> pd.DataFrame:
    df_fill = df.interpolate(limit_are='outside', limit=10)
    df_fill_n_drop = df_fill.dropna()

    if visualize:
        ax_raw = msno.matrix(df, figsize=(12, 6))
        ax_raw.set_title(f'[{df_nm}] Missing value before drop and fill na', fontsize=20)

        ax_fill = msno.matrix(df_fill, figsize=(12, 6))
        ax_fill.set_title(f'[{df_nm}] Missing value After fill na', fontsize=20)

        ax_fill_n_drop = msno.matrix(df_fill_n_drop, figsize=(12, 6))
        ax_fill_n_drop.set_title(f'[{df_nm}]Missing value After fill and drop na', fontsize=20)

    return df_fill_n_drop

df_19 = fill_missing_and_visualize(df_19, 'KAU-19')
df_20 = fill_missing_and_visualize(df_20, 'KAU-20')

In [ ]:
# plot - Check data by simple line plot
def viz_line_by_col(df):
    num_col = len(df.columns)

    df.plot(
        subplots=True, 
        layout=(int(np.ceil(num_col*2/3)), int(np.ceil(num_col*1/3))),
        figsize=(2*int(np.ceil(num_col*2/3)), 4*int(np.ceil(num_col*1/3))),
        )

for df in [df_19, df_20]:
    viz_line_by_col(df)

In [ ]:
# plot - Log transformation for scaling and visualization
def log_transform_n_viz(df, visualize=True):
    df_log_transformed = df.applymap(np.log)
    num_col = len(df_log_transformed.columns)

    if visualize:
        df_log_transformed.plot(
        subplots=True, 
        layout=(int(np.ceil(num_col*2/3)), int(np.ceil(num_col*1/3))),
        figsize=(2*int(np.ceil(num_col*2/3)), 4*int(np.ceil(num_col*1/3))),
        )
    
    return df_log_transformed

df_19 = log_transform_n_viz(df_19)
df_20 = log_transform_n_viz(df_20)

In [ ]:
# plot - visualize each time series data by lab plot and stationarity test
def viz_stationarity_check(df:pd.DataFrame):
    fig_lag_plot, ax_lag_plot = plt.subplots(len(df.columns), 2, figsize=(10, 5*len(tot_col_nm)))

    for idx, col_nm in enumerate(df.columns):
        
        pd.plotting.lag_plot(df[col_nm].dropna(), ax=ax_lag_plot[idx, 0])
        r_adf = adfuller(df[col_nm].dropna())
        r_kpss = kpss(df[col_nm].dropna())
        if any([r_adf[1] >= 0.05, r_kpss[1] < 0.05]):
            diff0_stationarity = 'Non-Stationary'
        else:
            diff0_stationarity = 'Stationary'
        ax_lag_plot[idx, 0].set_title(
            f'{col_nm} || No diff\nADF[{r_adf[1]:.2}] || KPSS [{r_kpss[1]:.2}]\n[{diff0_stationarity}]',
        fontsize=15, family='bold')

        pd.plotting.lag_plot(df[col_nm].diff().dropna(), ax=ax_lag_plot[idx, 1])
        r_adf = adfuller(df[col_nm].diff().dropna())
        r_kpss = kpss(df[col_nm].diff().dropna())
        if any([r_adf[1] < 0.05, r_kpss[1] >= 0.05]):
            diff1_stationarity = 'Stationary'
        else:
            diff1_stationarity = 'Non-Stationary'
        ax_lag_plot[idx, 1].set_title(
            f'{col_nm} || Diff 1\nADF[{r_adf[1]:.2}] || KPSS [{r_kpss[1]:.2}]\n[{diff1_stationarity}]', 
        fontsize=15, family='bold')

    fig_lag_plot.tight_layout()

for df in [df_19, df_20]:
    viz_stationarity_check(df)

In [ ]:
# Difference total DataFrame
df_19 = df_19.diff().dropna()
df_20 = df_20.diff().dropna()

In [ ]:
# plot - Check data by simple line plot
for df in [df_19, df_20]:
    viz_line_by_col(df)

# Save preprocessed data 

In [ ]:
dfs = [df_19, df_20]
df_nms = ['kau19', 'kau20']

for df, df_nm in zip(dfs, df_nms):
    df.to_csv(f'data/preprocessed/{df_nm}.csv')